# FastF1 Data Processing

In [2]:
#%pip install fastf1 pandas scikit-learn torch
import fastf1
import pandas as pd
import torch

In [10]:
# event_schedule -> event -> session -> laps -> telemetry
def process_event(event, year):
    fastest_qualifying_lap_times = {}

    qualifying = event.get_qualifying()
    qualifying.load()

    for driver in pd.unique(qualifying.results["Abbreviation"]):
        fastest_qualifying_lap_times[driver] = qualifying.laps.pick_driver(driver).pick_fastest().LapTime

    race = event.get_race()
    race.load()

    race_laps = race.laps.reset_index(drop=True)
    weather_data = race.laps.get_weather_data().reset_index(drop=True)
    race_laps = pd.concat([race_laps, weather_data.loc[:, ~(weather_data.columns == 'Time')]], axis=1)

    race_laps = race_laps[[
        "LapTime",
        "Driver",
        "LapNumber", # TODO does this need to be normalized
        "Stint",
        "Compound",
        "TyreLife",
        "Team",
        "TrackStatus",
        "Position",
        "Rainfall",
        "AirTemp",
        "TrackTemp",
        "PitInTime",
        "PitOutTime"
    ]]

    # Process pit in and out time
    race_laps["InLap"] = (~pd.isnull(race_laps["PitInTime"])).astype(int)
    race_laps["OutLap"] = (~pd.isnull(race_laps["PitOutTime"])).astype(int)
    race_laps = race_laps.drop(columns=["PitInTime", "PitOutTime"])

    # Qualifying results
    race_laps["FastestQualifyingLapTime"] = race_laps.apply(lambda row: fastest_qualifying_lap_times[row["Driver"]], axis=1)
    race_laps["FastestQualifyingLapTime"] = race_laps["FastestQualifyingLapTime"].dt.total_seconds() * 1000

    # Drop missing avlues
    race_laps = race_laps.dropna()

    # Normalize as int
    race_laps["FastestQualifyingLapTime"] = race_laps["FastestQualifyingLapTime"].astype(int)
    race_laps["LapTime"] = race_laps["LapTime"].dt.total_seconds() * 1000
    race_laps["LapTime"] = race_laps["LapTime"].astype(int)
    race_laps["LapNumber"] = race_laps["LapNumber"].astype(int)
    race_laps["Stint"] = race_laps["Stint"].astype(int)
    race_laps["TyreLife"] = race_laps["TyreLife"].astype(int)
    race_laps["Rainfall"] = race_laps["Rainfall"].astype(int)

    # Circuit and year
    race_laps["Event"] = event.EventName
    race_laps["Year"] = year


    return race_laps


event = fastf1.get_event(2022, 'Bahrain')
data = process_event(event, 2022)
data.to_csv("bahrain-2022.csv")

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '44', '77', '20', '14', '63', '10', '31', '47', '4', '23', '24', '22', '27', '3', '18', '6']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.0]
req            INFO 	Using cached da

In [11]:
data = pd.DataFrame()
num_events = 0
for year in range(2019, 2024):
    event_schedule = fastf1.get_event_schedule(year, include_testing=False)

    event_round = 0
    while True:
        event_round += 1

        try:
            event = event_schedule.get_event_by_round(event_round)
            event_data = process_event(event, year)

            data = pd.concat([data, event_data])
            num_events += 1
        except:
            break

data.to_csv("event-data.csv")
print(f"Processed {num_events} events")

core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '77', '5', '33', '16', '8', '20', '4', '7', '11', '27', '3', '23', '99', '26', '18', '10', '55', '63', '88']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.3.0]
req            INFO 	Using cach

Processed 104 events


In [12]:
event_data = pd.read_csv("event-data.csv").iloc[:, 1:]
processed_data = event_data

# One-hot encode driver
processed_data["Driver_Orig"] = processed_data["Driver"]
processed_data = pd.get_dummies(processed_data, columns=["Driver"], dtype=int)

# One-hot encode team
processed_data = pd.get_dummies(processed_data, columns=["Team"], dtype=int)

# One-hot encode position
processed_data = pd.get_dummies(processed_data, columns=["Position"], dtype=int)

# One-hot encode compound
processed_data = pd.get_dummies(processed_data, columns=["Compound"], dtype=int)

# One-hot encode event
processed_data["Event_Orig"] = processed_data["Event"]
processed_data = pd.get_dummies(processed_data, columns=["Event"], dtype=int)

# Encode track status
def track_status_apply(row):
    track_status = str(row["TrackStatus"])

    clear = "1" in track_status
    yellow = "2" in track_status
    safety_car = "4" in track_status
    red_flag = "5" in track_status
    virtual_safety_car_deployed = "6" in track_status
    virtual_safety_car_ending = "7" in track_status

    encoding = [clear, yellow, safety_car, red_flag, virtual_safety_car_deployed, virtual_safety_car_ending]
    return [float(i) for i in encoding]

processed_data[[
    "TrackStatus_Clear",
    "TrackStatus_YellowFlag",
    "TrackStatus_SafetyCar",
    "TrackStatus_RedFlag",
    "TrackStatus_VirtualSafetyCarDeployed",
    "TrackStatus_VirtualSafetyCarEnding"
]] = processed_data.apply(track_status_apply, axis=1, result_type="expand")
processed_data = processed_data.drop(columns=["TrackStatus"])

# Save processed data
processed_data.to_csv("processed-data.csv")

In [15]:
df = pd.read_csv("processed-data.csv").iloc[:, 1:]
    
def generate_ngrams(df, n, columns_to_drop=['Driver_Orig', 'Event_Orig']):
    """generate_ngrams creates n-grams from the input dataframe of consecutive laps
    from a driver at a single event.

    columns_to_drop: list of columns to drop from the n-gram (not tensor friendly)

    Returns a Pandas dataframe of the n-grams
    """
    ngrams_list = []

    df = df.sort_values(by=['Year', 'Event_Orig', 'Driver_Orig', 'LapNumber'])
    grouped = df.groupby(['Year', 'Event_Orig', 'Driver_Orig'])

    for _, group in grouped:
        for i in range(len(group) - n + 1):
            potential_ngram = group.iloc[i:i + n]

            lap_numbers = potential_ngram['LapNumber']
            min_lap_number = lap_numbers.min()
            if lap_numbers.tolist() == list(range(min_lap_number, min_lap_number + n)):
                ngrams_list.append(potential_ngram.values.flatten())


    col_names = [f'{col}_{i+1}' for i in range(n) for col in group]
    ngrams_df = pd.DataFrame(ngrams_list, columns=col_names)

    if columns_to_drop:
        all_columns_to_drop = [f'{col}_{i+1}' for i in range(n) for col in columns_to_drop]
        ngrams_df.drop(columns=all_columns_to_drop, errors='ignore', inplace=True)

    return ngrams_df


ngrams_df = generate_ngrams(df, 20)
tensor = torch.tensor(ngrams_df.values, dtype=torch.float32)
torch.save(tensor, "ngrams.pt")

In [21]:
# ngrams_df = pd.read_csv("ngrams.csv").iloc[:, 1:]
# tensor = torch.tensor(ngrams_df.values, dtype=torch.float32)
# print(tensor.shape)

True
